In [ ]:
################################################################################################
################################################################################################
## REVISIÓN DE VERSIONES                                                                      ##
##                                                                                            ##
## v.2.1                                                                                      ##
## - añade la paginación de los partidos diarios                                              ##
##                                                                                            ##
## v.2.0                                                                                      ##
## - scraping de una página específica de partidos diarios                                    ##
## - crea dataframe y lo envía a archivo .csv                                                 ##
##                                                                                            ##
################################################################################################
################################################################################################

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
def obtener_stats(func_stats):
    stats_dict = {}
    
    date = func_stats.find('div', attrs={'class':'info match-link'})
    if date:
        stats_dict['date'] = date.get('starttime').replace('T',' ').replace(':00+00:00','')
    else:
        stats_dict['date'] = '-'
 
    draw = func_stats.find('div', attrs={'class':'content-box draw'}).find('div', attrs={'class':'cont'})
    if draw:
        stats_dict['draw'] = draw.get_text('/',strip=True).replace('%','')[4:10]
    else:
        stats_dict['draw'] = '-' 
        
    homeRank = func_stats.find('div', attrs={'class':'position t1'})
    if homeRank:
        stats_dict['homeRank'] = homeRank.get_text()
    else:
        stats_dict['homeRank'] = '-'    
            
    homeTeam = func_stats.find('p', attrs={'itemprop':'homeTeam'})
    if homeTeam:
        stats_dict['homeTeam'] = homeTeam.get_text('/', strip=True)
    else:
        stats_dict['homeTeam'] = '-'

    homeGoals = func_stats.find('div', attrs={'class':'content-box t-1'}).find('div', attrs={'class':'poss-box'})
    if homeGoals:
        stats_dict['homeGoals'] = homeGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')
    else:
        stats_dict['homeGoals'] = '-'     
        
    awayGoals = func_stats.find('div', attrs={'class':'content-box t-2'}).find('div', attrs={'class':'poss-box'})
    if awayGoals:
        stats_dict['awayGoals'] = awayGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')
    else:
        stats_dict['awayGoals'] = '-'     

    awayTeam = func_stats.find('p', attrs={'itemprop':'awayTeam'})
    if awayTeam:
        stats_dict['awayTeam'] = awayTeam.get_text('/', strip=True)
    else:
        stats_dict['awayTeam'] = '-'     
     
    awayRank = func_stats.find('div', attrs={'class':'position t2'})
    if awayRank:
        stats_dict['awayRank'] = awayRank.get_text()
    else:
        stats_dict['awayRank'] = '-' 
        
    competition = func_stats.find('h3', attrs={'class':'competition'})
    if competition:
        stats_dict['competition'] = competition.get_text('/', strip=True)
    else:
        stats_dict['competition'] = '-'
   
    return stats_dict

In [3]:
df = pd.DataFrame()

In [4]:
#url = input('Introduce la url a scrapear: ')
#url = 'https://es.besoccer.com/livescore/2022-09-21'

urlppal = 'https://es.besoccer.com/livescore/2022-09-25'
for i in range(1,6):
    url = urlppal+f'/{i}'
    besoccer = requests.get(url)
    besoccer.status_code
    s_besoccer = BeautifulSoup(besoccer.text, 'lxml')
    matches = s_besoccer.find('div', attrs={'class':'matches'}).select('a[href*="/partido/"]')
    [match.text for match in matches] 
    matches
    try:
        match = matches[0]
        match.get('href')
    except Exception as e:
        print('No hay más partidos que analizar')

    links_matches = [match.get('href') for match in matches]
    for i in range(len(links_matches)):
        list_analisis = []
        r_match = requests.get(links_matches[i])
        s_match = BeautifulSoup(r_match.text, 'lxml')
        find_analisis = s_match.find('div', attrs={'class':'menu-scroll'}).select('a[href*="analisis"]')
        try:
            analisis = find_analisis[0]
            link_analisis = analisis.get('href')
            r_link_analisis = requests.get(link_analisis)
            s_link_analisis = BeautifulSoup(r_link_analisis.text, 'lxml')
            df = df.append(obtener_stats(s_link_analisis), ignore_index = True)
            print(i,'.- Scrapeando',url)
            print(i,'.-', link_analisis)
        except Exception as e:
            print(i,'.- La url ',links_matches[i],' no tiene sección análisis')
            print(i,'.-',e)
i += 1


C:\Users\jesus\AppData\Local\Temp\ipykernel_1688\4279837899.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


0 .- Scrapeando https://es.besoccer.com/livescore/2022-09-25/1
0 .- https://es.besoccer.com/partido/seleccion-andorra/seleccion-letonia/2023124/analisis


C:\Users\jesus\AppData\Local\Temp\ipykernel_1688\4279837899.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


1 .- Scrapeando https://es.besoccer.com/livescore/2022-09-25/1
1 .- https://es.besoccer.com/partido/seleccion-moldavia/seleccion-liechtenstein/2023157/analisis


C:\Users\jesus\AppData\Local\Temp\ipykernel_1688\4279837899.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


...

C:\Users\jesus\AppData\Local\Temp\ipykernel_1688\4279837899.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


3 .- Scrapeando https://es.besoccer.com/livescore/2022-09-25/1
3 .- https://es.besoccer.com/partido/azerbaiyan/seleccion-kazakhistan/2023214/analisis


C:\Users\jesus\AppData\Local\Temp\ipykernel_1688\4279837899.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


4 .- Scrapeando https://es.besoccer.com/livescore/2022-09-25/1
4 .- https://es.besoccer.com/partido/seleccion-holanda/seleccion-belgica/202357/analisis


KeyboardInterrupt: 

In [ ]:
df.to_csv('matches.csv', header = False)

In [ ]:
df.style.set_properties(**{'text-align': 'left'}) 